In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

## Determine if the MRI slice orientation is axial, sagittal or coronal

Thanks to this ([notebook](https://www.kaggle.com/davidbroberts/determining-mr-slice-orientation/notebook)) by @davidbroberts, I learned that the orientation can be derived from the `ImageOrientationPatient` column of dicom metadata.

Since the orientation can be derived from the `ImageOrientationPatient` column of the dicom metadata, we only need to convert the values from string to float to integer rounded to the nearest digit. Then the resulting image vectors can be mapped into axial, sagittal, or coronal using the table below. Thanks to this ([forum](https://stackoverflow.com/questions/34782409/understanding-dicom-image-attributes-to-get-axial-coronal-sagittal-cuts)). 

| Orientation | ImageOrientationPatient |
| --- | --- |
| Axial | [1, 0, 0, 0, 1, 0] |
| Sagittal | [0, 1, 0, 0, 0,-1] |
| Coronal | [1, 0, 0, 0, 0,-1] |


In [ ]:
def mri_orientation(index):
    '''
    Determine the MRI orientation (axial, sagittal, coronal) based on the 
    `ImageOrientationPatient` column of the dicom metadata. -@rickandjoe
    reference: https://stackoverflow.com/questions/34782409/understanding-dicom-image-attributes-to-get-axial-coronal-sagittal-cuts
    '''
    
    image_vectors = train_metadata['ImageOrientationPatient'].iloc[index].replace('[','').replace(']','').split(',')
    image_vectors = [round(float(x)) for x in image_vectors]

    #[1,0,0,0,1,0]
    if image_vectors[0] == 1 and image_vectors[4] == 1: 
        return 'axial'
    
    #[0,1,0,0,0,-1]
    elif image_vectors[1] == 1 and image_vectors[5] == -1: 
        return 'sagittal'
    
    #[1,0,0,0,0,-1]
    elif image_vectors[0] == 1 and image_vectors[5] == -1: 
        return 'coronal'
    
    else:
        return 'unknown'

In [ ]:
train_metadata = pd.read_csv('../input/extract-metadata-from-dicom/dicom_meta_train.csv', 
                       usecols=['BraTS21ID', 'dataset', 'type', 'dicom_src', 'ImageOrientationPatient'])

### Include in Train Metadata

Create another column "Orientation" in our train metadata dataframe.

In [ ]:
orientation = []
for index in range(len(train_metadata)):
    orientation.append(mri_orientation(index))
    
train_metadata['Orientation'] = orientation
train_metadata

In [ ]:
print(train_metadata['Orientation'].value_counts())
with plt.xkcd():
    sns.countplot(data=train_metadata, x='Orientation', order=['axial', 'sagittal', 'coronal'])

But since there are multiple images per viewtype per case id, the distribution of MRI orientation could be different if we only consider one image per viewtype per case id.

In [ ]:
unique_df = train_metadata[['BraTS21ID','type','Orientation']]
unique_df['BraTS21ID+type'] = unique_df['BraTS21ID'].astype(str) + unique_df['type'].astype(str)
# Drop all the other images with the same viewtype and case id. Only retain the first image.
unique_df = unique_df.drop_duplicates(subset='BraTS21ID') 
unique_df[['BraTS21ID+type','Orientation']]

We can see in the plot below that in the same order, most images have axial orientation followed by sagittal orientation and lastly, very few images (22 in train set) have coronal orientation.

In [ ]:
print(unique_df['Orientation'].value_counts())
with plt.xkcd():
    sns.countplot(data=unique_df, x='Orientation', order=['axial', 'sagittal', 'coronal'])

### Examples

Let's visualize some examples on images with the most nonzero pixels per viewtype (T1w, T1wCE, T2w, FLAIR) per case id.

In [ ]:
train_df = pd.read_csv('../input/images-with-the-most-nonzero-pixels/train_df.csv')
train_df['caseID'] = train_df['caseID'].astype(str).str.zfill(5)

In [ ]:
train_df

In [ ]:
#case_ids = ['00000','00002','00003','00005','00006']
case_ids = train_df['caseID'].iloc[np.random.randint(len(train_df), size=5)]

for case_id in case_ids:
    fig = plt.figure(figsize=(26,7.5))
    fig.suptitle(f'Case ID: {case_id} \nMGMT_value = {train_df[train_df.caseID==case_id].MGMT_value.iloc[0]}', fontsize=30)
    axis = 141
    plt.gray()
    for viewtype in ['T1w','T1wCE','T2w','FLAIR']:
        filename = train_df[train_df.caseID==case_id][f'{viewtype}'].iloc[0]
        img = plt.imread(f'../input/rsna-miccai-png/train/{case_id}/{viewtype}/{filename}')
        filename = filename.replace('png','dcm')
        orientation = train_metadata[train_metadata.dicom_src == f'./train/{case_id}/{viewtype}/{filename}'].Orientation.iloc[0]

        ax = fig.add_subplot(axis)
        plt.title(f'{viewtype} - {orientation}', fontsize=20)
        plt.imshow(img, aspect='auto')
        plt.tight_layout()
        axis = axis + 1


# Test Data

Let's do the same thing for the test set and hopefully we can find similar patterns and distributions.

In [ ]:
test_metadata = pd.read_csv('../input/extract-metadata-from-dicom/dicom_meta_test.csv', 
                       usecols=['BraTS21ID', 'dataset', 'type', 'dicom_src', 'ImageOrientationPatient'])

In [ ]:
orientation = []
for index in range(len(test_metadata)):
    orientation.append(mri_orientation(index))
    
test_metadata['Orientation'] = orientation
test_metadata

In [ ]:
print(test_metadata['Orientation'].value_counts())
with plt.xkcd():
    sns.countplot(data=test_metadata, x='Orientation', order=['axial', 'sagittal', 'coronal'])

In [ ]:
unique_df = test_metadata[['BraTS21ID','type','Orientation']]
unique_df['BraTS21ID+type'] = unique_df['BraTS21ID'].astype(str) + unique_df['type'].astype(str)
# Drop all the other images with the same viewtype and case id. Only retain the first image.
unique_df = unique_df.drop_duplicates(subset='BraTS21ID') 
unique_df[['BraTS21ID+type','Orientation']]

In [ ]:
print(unique_df['Orientation'].value_counts())
with plt.xkcd():
    sns.countplot(data=unique_df, x='Orientation', order=['axial', 'sagittal', 'coronal'])

In [ ]:
test_df = pd.read_csv('../input/images-with-the-most-nonzero-pixels/test_df.csv')
test_df['caseID'] = test_df['caseID'].astype(str).str.zfill(5)

In [ ]:
test_df

In [ ]:
#case_ids = ['00001','00013','00015','00027','00037']
case_ids = test_df['caseID'].iloc[np.random.randint(len(test_df), size=5)]

for case_id in case_ids:
    fig = plt.figure(figsize=(26,7.5))
    fig.suptitle(f'Case ID: {case_id} \nMGMT_value = ?', fontsize=30)
    axis = 141
    plt.gray()
    for viewtype in ['T1w','T1wCE','T2w','FLAIR']:
        filename = test_df[test_df.caseID==case_id][f'{viewtype}'].iloc[0]
        img = plt.imread(f'../input/rsna-miccai-png/test/{case_id}/{viewtype}/{filename}')
        filename = filename.replace('png','dcm')
        orientation = test_metadata[test_metadata.dicom_src == f'./test/{case_id}/{viewtype}/{filename}'].Orientation.iloc[0]

        ax = fig.add_subplot(axis)
        plt.title(f'{viewtype} - {orientation}', fontsize=20)
        plt.imshow(img, aspect='auto')
        plt.tight_layout()
        axis = axis + 1



In [ ]:
train_metadata.to_csv('train_metadata_mri_orientation.csv', index=False)
test_metadata.to_csv('test_metadata_mri_orientation.csv', index=False)